In [1]:
from itertools import combinations

def unify(x, y, theta=None):
    if theta is None:
        theta = {}

    if x == y:
        return theta
    elif isinstance(x, str) and x.islower():
        return unify_var(x, y, theta)
    elif isinstance(y, str) and y.islower():
        return unify_var(y, x, theta)
    elif isinstance(x, tuple) and isinstance(y, tuple) and len(x) == len(y):
        return unify(x[1:], y[1:], unify(x[0], y[0], theta))
    return None

def unify_var(var, value, theta):
    if var in theta:
        return unify(theta[var], value, theta)
    elif value in theta:
        return unify(var, theta[value], theta)
    else:
        theta[var] = value
        return theta

def resolve(clause1, clause2):
    resolvents = set()

    for pred1 in clause1:
        for pred2 in clause2:
            theta = unify(pred1, negate(pred2))
            if theta is not None:
                resolvent = (substitute(clause1, theta) | substitute(clause2, theta)) - {pred1, pred2}
                resolvents.add(frozenset(resolvent))
    return resolvents

def negate(predicate):
    if isinstance(predicate, tuple) and predicate[0] == 'not':
        return predicate[1]
    return ('not', predicate)

def substitute(clause, theta):
    return {substitute_predicate(p, theta) for p in clause}

def substitute_predicate(predicate, theta):
    if isinstance(predicate, str):
        return theta.get(predicate, predicate)
    elif isinstance(predicate, tuple):
        return (predicate[0],) + tuple(theta.get(arg, arg) for arg in predicate[1:])
    return predicate

def resolution(kb, query):
    negated_query = frozenset({negate(query)})
    clauses = kb | {negated_query}
    new_clauses = set()

    while True:
        for clause1, clause2 in combinations(clauses, 2):
            resolvents = resolve(clause1, clause2)
            if frozenset() in resolvents:
                return True
            new_clauses.update(resolvents)

        if new_clauses.issubset(clauses):
            return False

        clauses |= new_clauses

kb = {
    frozenset({('Mother', 'leela', 'Oshin')}),
    frozenset({('Alive', 'leela')}),
    frozenset({('not', ('Mother', 'x', 'y')), ('Parent', 'x', 'y')}),
    frozenset({('not', 'Parent', 'x', 'y'), ('not', 'Alive', 'x'), ('Older', 'x', 'y')}),
}

query = ('Older', 'leela', 'Oshin')

result = resolution(kb, query)
if result:
    print("Proven: Leela is older than Oshin.")
else:
    print("Cannot prove: Leela is older than Oshin.")
print("1BM22CS028")
print("AKASH K S")

Proven: Leela is older than Oshin.
1BM22CS028
AKASH K S
